In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('alura').getOrCreate()


dataframe_site = spark.read.json('site.json')
dataframe_loja = spark.read.format('orc').load('loja.orc')
dataframe_contabilidade = spark.read.format('csv').option('header', True).option("delimiter", ";").load("contabilidade.csv.gz")


dataframe_contabilidade

dataframe_site.show()
dataframe_loja.show()
dataframe_contabilidade.show()

In [ ]:
from pyspark.sql.types import *

schema = StructType([StructField("id", IntegerType(), True),
                     StructField("status", StringType(), True),
                     StructField("data_pedido", TimestampType(), True)])

dataframe_contabilidade = spark.read.format('csv').schema(schema).option('header', True).option("delimiter", ";").load("contabilidade.csv.gz")


dataframe_contabilidade.printSchema()

dataframe_inner = dataframe_contabilidade.join(dataframe_site, dataframe_contabilidade.id == dataframe_site.id_ped)
dataframe_inner.count()
dataframe_inner.show()

dataframe_site_loja = dataframe_loja.join(dataframe_site, dataframe_loja.id_pedido == dataframe_site.id_ped, how='full')
dataframe_site_loja.show()
dataframe_site_loja.count()

dataframe_site_loja_dis = dataframe_site_loja.distinct()
dataframe_site_loja_dis.count()


In [ ]:
import pyspark.sql.functions as F

dataframe_contabilidade.filter(dataframe_contabilidade.status == 'ENT').show(30, False)

dataframe_fevereiro = dataframe_contabilidade.filter(dataframe_contabilidade.status == 'ENT')
dataframe_fevereiro

dataframe_fevereiro.filter(dataframe_fevereiro.data_pedido >= '2019-02-01').filter(dataframe_fevereiro.data_pedido <= '2019-02-28').show()

dataframe_fevereiro = dataframe_fevereiro.filter(dataframe_fevereiro.data_pedido >= '2019-02-01').filter(dataframe_fevereiro.data_pedido <= '2019-02-28')
dataframe_fevereiro.show()
dataframe_fevereiro.count()

dataframe_site_loja

dataframe_groupby = dataframe_site_loja.groupBy(dataframe_site_loja.id_pedido).agg(F.sum(dataframe_site_loja.preco_venda).alias('preco_venda_agrupado'))

dataframe_groupby.count()
dataframe_site_loja.count()
dataframe_groupby.show()